In [ ]:
%pip install git+https://github.com/Louis-Li-dev/ML-Toolkit

  Cloning https://github.com/Louis-Li-dev/ml-toolkit to c:\users\ss348\appdata\local\temp\pip-req-build-z7p034he
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none -q https://github.com/Louis-Li-dev/ml-toolkit 'C:\Users\ss348\AppData\Local\Temp\pip-req-build-z7p034he'
  remote: Repository not found.
  fatal: repository 'https://github.com/Louis-Li-dev/ml-toolkit/' not found
ERROR: Command errored out with exit status 128: git clone --filter=blob:none -q https://github.com/Louis-Li-dev/ml-toolkit 'C:\Users\ss348\AppData\Local\Temp\pip-req-build-z7p034he' Check the logs for full command output.


In [4]:

from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
